# Main PyEPO
This notebook uses our classes in close connection with the PyEPO library and follows their tutorial in creating a shortest path optnet.

## Optimizers
We start with defining an optimization problem with an optimizer.

In [ ]:
# Import numpy and ShortestPathGrb class
import pyepo
import numpy as np
from torch import optim
from sklearn.model_selection import train_test_split

import dflintdpy
from dflintdpy.models.grid import Grid
from dflintdpy.solvers.shortest_path_grb import ShortestPathGrb
from dflintdpy.predictors.linear_regression import LinearRegression
from dflintdpy.utils.dfl_trainer import DFLTrainer
from dflintdpy.data.adverse.adverse_loader import AdvLoader

In [ ]:
# Set parameters
m, n = (5, 5)
np.random.seed(42)  # for reproducibility

# Create grid instance
grid = Grid(m, n)

# Create a opt_model instance
opt_model = ShortestPathGrb(grid)

In [ ]:
# Create a random cost array for the grid
cost = np.arange((m-1)*n + m*(n-1))
np.random.shuffle(cost)

# Set the cost for the grid (Optionally specify the source and target nodes)
opt_model.setObj(cost)

In [ ]:
# Solve shortest path problem
path, obj = opt_model.solve(visualize=True)

## Datasets
We use PyEPO to generate data for the shortest path problem and use its ``optDataset`` class for data storage and loading.

In [ ]:
# Set parameters for data generation
num_train_data = 1000 # number of training data
num_test_data = 1000 # number of test data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width

# Generate data for shortest path problem
feats, costs = pyepo.data.shortestpath.genData(
    num_train_data+num_test_data, 
    num_feat, 
    (m,n), 
    deg=deg, 
    noise_width=e, 
    seed=135
)

In [ ]:
# split train test data
x_train, x_test, c_train, c_test = train_test_split(
    feats, 
    costs, 
    test_size=num_test_data, 
    random_state=42
)

In [ ]:
# Create datasets for training and testing
dataset_train = dflintdpy.data.adverse.adverse_dataset.generate_opt_dataset(opt_model, x_train, c_train)
dataset_test = dflintdpy.data.adverse.adverse_dataset.generate_opt_dataset(opt_model, x_test, c_test)

In [ ]:
# Wrap dataset into PyTorch DataLoader
batch_size = 32
loader_train = AdvLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = AdvLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Training and Testing
We will now create a predictive model. Then we train and test it with the artificial data created in the previous section.

In [ ]:
# Instantiate linear regression model
model = LinearRegression(num_feat=num_feat, num_edges=opt_model.num_cost)

In [ ]:
# Init SPO+ loss
spop = pyepo.func.SPOPlus(opt_model, processes=1)

# Init optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-2)

### Train SPO+ loss
We will now train the model with SPO+ loss and visualize the learning curves. Note that we do not have to instantiate the linear regression in this instance as it has already been instantiated previously.

In [ ]:
# Set the number of epochs for training
epochs = 5

# Create a trainer instance
trainer = DFLTrainer(
    pred_model=model, 
    opt_model=opt_model, 
    optimizer=optimizer, 
    loss_fn=spop
)

train_loss_log, train_regret_log, test_loss_log, test_regret_log = trainer.fit(loader_train, loader_test, epochs=epochs)

In [ ]:
DFLTrainer.vis_learning_curve(
    trainer,
    train_loss_log,
    train_regret_log,
    test_loss_log,
    test_regret_log
)

print("Final regret on test set: ", test_regret_log[-1])

### Differentiable Black Box Trainer
We will now train the Black Box trainer to compare the different performances. Note that the best comparison is the regret, as it is calculated independent of the chosen loss model.

In [ ]:
# Instantiate new linear regression model
model = LinearRegression(num_feat=num_feat, num_edges=opt_model.num_cost)

In [ ]:
# Init dbb solver
dbb = pyepo.func.blackboxOpt(opt_model, lambd=20)
# Set loss
from torch import nn
l1 = nn.L1Loss()

# Loss function
def dbbl1(cp, c, z):
    # Black-box optimizer
    wp = dbb(cp)
    # Objective value
    zp = (wp * c).sum(1).view(-1, 1)
    # Loss
    loss = l1(zp, z)
    return loss

# Init optimizer
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
# Create a trainer instance
trainer = Trainer(pred_model=model, 
                  opt_model=opt_model, 
                  optimizer=optimizer, 
                  loss_fn=dbbl1, 
                  method_name="dbb"
               )

# Train the model with DBB loss
train_loss_log, train_regret_log, test_loss_log, test_regret_log = trainer.fit(loader_train, loader_test, epochs=epochs)

In [ ]:
trainer.vis_learning_curve(
    trainer,
    train_loss_log,
    train_regret_log,
    test_loss_log,
    test_regret_log
)

print("Final regret on test set: ", test_regret_log[-1])